In [1]:
# 6.6 使用Python實作梯度提升的完整機制(迴歸)

In [2]:
# --- 第 1 部分 ---
# 載入函式庫與資料集
from copy import deepcopy
from sklearn.datasets import load_diabetes
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
import numpy as np
diabetes = load_diabetes()
train_size = 400
train_x, train_y = diabetes.data[:train_size], diabetes.target[:train_size]
test_x, test_y = diabetes.data[train_size:], diabetes.target[train_size:]
np.random.seed(123456)


In [3]:
# --- 第 2 部分 ---
# 設定總體大小、學習率及決策樹深度
ensemble_size = 200
learning_rate = 0.1
base_classifier = DecisionTreeRegressor(max_depth=3)

# 創建變數以儲存基學習器與其預測
base_learners = []
# 初始預測為標籤平均值
previous_p = np.zeros(len(train_y)) + np.mean(train_y)


In [7]:
# --- 第 3 部分 ---
# 訓練弱學習器
for _ in range(ensemble_size):
    # 計算偽殘差
    errors = train_y - previous_p
    # 訓練弱學習器
    learner = deepcopy(base_classifier)
    learner.fit(train_x, errors)
    # 在訓練集上預測偽殘差
    p = learner.predict(train_x)
    # 將預測值乘上學習率
    # 再加上先前的預測值
    previous_p = previous_p + learning_rate * p
    # 儲存弱學習器
    base_learners.append(learner)


In [8]:
# --- 第 4 部分 ---
# 評估集成後效能
# 訓練資料集的標籤平均值為初始預測
previous_p = np.zeros(len(test_y)) + np.mean(train_y)
# 每個弱學習器都用測試資料集的特徵來預測偽殘差
# 預測值乘上學習率後相加
for learner in base_learners:
    p = learner.predict(test_x)
    previous_p = previous_p + learning_rate * p


In [9]:
# --- 第 5 部分 ---
# 顯示結果
r2 = metrics.r2_score(test_y, previous_p)
mse = metrics.mean_squared_error(test_y, previous_p)
print('Gradient Boosting:')
print('R-squared: %.2f' % r2)
print('MSE: %.2f' % mse)


Gradient Boosting:
R-squared: 0.44
MSE: 3109.81
